In [1]:
import pandas as pd
df = pd.read_csv("anime_dataset_cleaned.csv")
df.head()

,Unnamed: 0,title_romaji,title_english,rating,description,genres,format,status,episodes,studios,coverImage_large
0,0,Sousou no Frieren,Frieren: Beyond Journey’s End,9.1,The adventure is over but life goes on for an ...,"Adventure, Drama, Fantasy",TV,FINISHED,28,"Toho, MADHOUSE, Shogakukan, Nippon Television ...",https://s4.anilist.co/file/anilistcdn/media/an...
1,1,Gintama°,Gintama Season 3,9.1,"Gintoki, Shinpachi, and Kagura return as the f...","Action, Comedy, Drama, Sci-Fi",TV,FINISHED,51,"Bandai Namco Pictures, Aniplex",https://s4.anilist.co/file/anilistcdn/media/an...
2,2,Gintama: THE FINAL,Gintama: THE VERY FINAL,9.1,Gintama: THE FINAL is the 3rd and final film a...,"Action, Comedy, Drama, Sci-Fi",MOVIE,FINISHED,1,"Bandai Namco Pictures, Shueisha, Aniplex, Warn...",https://s4.anilist.co/file/anilistcdn/media/an...
3,3,Hagane no Renkinjutsushi: FULLMETAL ALCHEMIST,Fullmetal Alchemist: Brotherhood,9.0,"""In order for something to be obtained, someth...","Action, Adventure, Drama, Fantasy",TV,FINISHED,64,"bones, Aniplex, Square Enix, Mainichi Broadcas...",https://s4.anilist.co/file/anilistcdn/media/an...
4,4,Chainsaw Man: Reze-hen,Chainsaw Man – The Movie: Reze Arc,9.0,Theatrical follow-up to Chainsaw Man.\nDenji b...,"Action, Drama, Horror, Romance, Supernatural",MOVIE,FINISHED,1,"MAPPA, Shueisha, Toho",https://s4.anilist.co/file/anilistcdn/media/an...


In [2]:
train_df = pd.DataFrame(df, columns=['title_romaji', 'description'])
train_df = train_df.set_index('title_romaji')
train_df.head()

,description
title_romaji,
Sousou no Frieren,The adventure is over but life goes on for an ...
Gintama°,"Gintoki, Shinpachi, and Kagura return as the f..."
Gintama: THE FINAL,Gintama: THE FINAL is the 3rd and final film a...
Hagane no Renkinjutsushi: FULLMETAL ALCHEMIST,"""In order for something to be obtained, someth..."
Chainsaw Man: Reze-hen,Theatrical follow-up to Chainsaw Man.\nDenji b...


lemmatization

In [3]:
import nltk
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\denis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\denis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\denis\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\denis\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

In [4]:
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()



def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return 'a'
    elif tag.startswith('V'):
        return 'v'
    elif tag.startswith('N'):
        return 'n'
    elif tag.startswith('R'):
        return 'r'
    else:
        return 'n'


def lemmatize_description(description):
  tokens = word_tokenize(description)
  tagged_tokens = pos_tag(tokens)
  lemmatized_sentence = []
  for word, tag in tagged_tokens:
      if word.lower() == 'are' or word.lower() in ['is', 'am']:
          lemmatized_sentence.append(word)
      else:
          lemmatized_sentence.append(
              lemmatizer.lemmatize(word, get_wordnet_pos(tag)))

  return ' '.join(lemmatized_sentence)

In [5]:
train_df['description'] = train_df['description'].apply(lambda x: lemmatize_description(x))

In [6]:
print(f"original: {df['description'][0]}")
print(f"lemmatized: {train_df['description'][0]}")

original: The adventure is over but life goes on for an elf mage just beginning to learn what living is all about. Elf mage Frieren and her courageous fellow adventurers have defeated the Demon King and brought peace to the land. But Frieren will long outlive the rest of her former party. How will she come to understand what life means to the people around her? Decades after their victory, the funeral of one her friends confronts Frieren with her own near immortality. Frieren sets out to fulfill the last wishes of her comrades and finds herself beginning a new adventure…

(Source: Crunchyroll)
lemmatized: The adventure is over but life go on for an elf mage just begin to learn what living is all about . Elf mage Frieren and her courageous fellow adventurer have defeat the Demon King and bring peace to the land . But Frieren will long outlive the rest of her former party . How will she come to understand what life mean to the people around her ? Decades after their victory , the funeral

tf-idf vectorization

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfvec = TfidfVectorizer()
desc_vect = tfidfvec.fit_transform((train_df["description"]))

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
import pickle

cos_sim = cosine_similarity(desc_vect, desc_vect)
pickle.dump(cos_sim, open('cosine.pkl', 'wb'))
print(cos_sim)

[[1.         0.06685613 0.12228712 ... 0.03890752 0.0588308  0.01163329]
 [0.06685613 1.         0.24562155 ... 0.04594224 0.09794908 0.03938421]
 [0.12228712 0.24562155 1.         ... 0.09106871 0.05175201 0.02627798]
 ...
 [0.03890752 0.04594224 0.09106871 ... 1.         0.06288368 0.05085848]
 [0.0588308  0.09794908 0.05175201 ... 0.06288368 1.         0.02362506]
 [0.01163329 0.03938421 0.02627798 ... 0.05085848 0.02362506 1.        ]]


In [9]:
from rapidfuzz import process
import pandas as pd
import numpy as np

indices = pd.Series(range(len(train_df)), index=train_df.index)
titles = train_df.index

def find_closest_title(user_title, all_titles, threshold=70):
    match = process.extractOne(user_title, all_titles)
    return match[0] if match and match[1] >= threshold else None


def recommendations(title, cosine_sim=cos_sim, range_nr=10, lambda_param=0.7, candidate_pool=50):
    real_title = find_closest_title(title, titles)

    if real_title is None:
        return []

    query_idx = indices[real_title]

    # Get similarity scores from cosine matrix
    sims = cosine_sim[query_idx]

    # Get top candidates (excluding itself)
    candidates = list(np.argsort(sims)[::-1][1:candidate_pool+1])

    selected = []

    while len(selected) < range_nr and candidates:
        mmr_scores = {}

        for c in candidates:
            relevance = sims[c]

            if selected:
                diversity = max(cosine_sim[c][s] for s in selected)
            else:
                diversity = 0

            mmr_scores[c] = lambda_param * relevance - (1 - lambda_param) * diversity

        next_best = max(mmr_scores, key=mmr_scores.get)
        selected.append(next_best)
        candidates.remove(next_best)

    return titles[selected].tolist()


In [10]:
recommendations("gintama")

['Gintama: Shinyaku Benizakura-hen',
 'Gintama: Jump Festa 2005 Special',
 'Sakura Quest',
 'Get Ride! AMDriver',
 'Black Jack (TV)',
 'Atarashii Joushi wa Dotennen',
 'Campione!: Matsurowanu Kamigami to Kamigoroshi no Maou',
 'GANGSTA.',
 'Hoozuki no Reitetsu 2',
 'Mouretsu Pirates: ABYSS OF HYPERSPACE - Akuu no Shinen']